# TimeRNN
- TimeRNNはT個のRNNレイヤを連結したネットワークである。
- 誤差逆伝播についての特徴を纏めると
  - 出力が2つに分岐しているため， 逆伝播では各勾配が合算されて伝わる（$dh_t (dy_t) + dh_{next}$）
  - 複数のRNNレイヤがあり，それらのRNNレイヤは同じ重みを使用しているため，最終的な重みの勾配は各RNNレイヤの重みの勾配を足し合わせたものになる



In [ ]:
# 参考: https://github.com/oreilly-japan/deep-learning-from-scratch-2/tree/master

import numpy as np

# 時系列としてレイヤーが並んでいないタイプのRNN
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b] # それぞれの重みとバイアスをリストとして渡される
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next) # 入力，隠れ層への入力，出力　をキャッシュしておく
        return h_next

    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2) # tanhの微分
        db = np.sum(dt, axis=0)
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, Wh.T)
        dWx = np.dot(x.T, dt) # 入力値×流れてきた誤差，dtが時系列t毎に後のレイヤから流れてきた誤差の違いにより値が異なる
        dx = np.dot(dt, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

# 時系列としてレイヤー並んでいるタイプのRNN
class TimeRNN:
    # statefulがTrueのときは隠れ状態を維持するようになる。
    # つまり，どんなに長い時系列データであったもTime RNNレイヤの
    # 順伝播を断ち切ることなく伝播させる（長い時系列データを処理するときは隠れ状態を維持する必要がある）
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h) # バッチサイズ, T個分の時系列, 入力ベクトルの次元数
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs

    # RNN例やの順伝播では出力が2つに分岐しているため，
    # 逆伝播では各勾配が合算されて伝わる（dh_t+dh_next）
    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx

            # 複数のRNNレイヤがあり，それらのRNNレイヤは同じ重みを使用しているため，
            # 最終的な重みの勾配は各RNNレイヤの重みの勾配を足し合わせたものになる
            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        # 最終的に各レイヤ毎の重みを保存
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None